In [5]:
import pandas as pd
import numpy as np
import matplotlib as plt

data_df = pd.read_csv("data.csv")

print(data_df.head(5))
print(data_df.describe())

   Product_Code Warehouse Product_Category       Date Order_Demand
0  Product_0993    Whse_J     Category_028  2012/7/27         100 
1  Product_0979    Whse_J     Category_028  2012/1/19         500 
2  Product_0979    Whse_J     Category_028   2012/2/3         500 
3  Product_0979    Whse_J     Category_028   2012/2/9         500 
4  Product_0979    Whse_J     Category_028   2012/3/2         500 
        Product_Code Warehouse Product_Category       Date Order_Demand
count        1048575   1048575          1048575    1037336      1048575
unique          2160         4               33       1729         3828
top     Product_1359    Whse_J     Category_019  2013/9/27        1000 
freq           16936    764447           481099       2075       112682


In [2]:
data_df.pivot_table(values="Order_Demand", index="Product_Category", aggfunc=[np.sum, np.mean, np.median])

TypeError: Could not convert 4 4 4 1 3 1 7 7 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 3 4 2 2 4 1 1 1 3 1 2 50 50 1 4 2 2 1 1 2 50 4 4 1 1 2 2 2 1 1 1 1 1 1 1 3 3 1 2 1 2 1 2 1 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 1 4 4 1 2 2 1 1 1 2 3 1 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 3 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 3 1 1 1 1 1 1 1 1 1 2 3 1 1 2 5 2 11 1 5 5 9 197 9 197 4 1 134 53 1 9 3 2 197 2 3 3 4 4 4 4 2 20 3 3 1 3 1 2 3 1 1 3 3 5 2 2 2 2 1 2 15 21 35 30 30 40 30 30 5 4 10 5 4 14 1 2 1 3 3 20 5 2 5 20 1 5 5 1 3 10 2 4 30 12 2 7 30 1 5 9 5 1 1 1 1 300 259 41 300 100 2 4 1 4 1 4 1 1 1 1 2 1 2 5 2 1 3 1 1 1 1 4 2 2 1 2 2 4 1 1 2 3 1 3 3 5 2 1 2 2 1 1 2 2 1 1 2 3 1 3 1 1 1 3 1 1 3 25 20 120 30 50 50 30 18 80 38 22 14 5 7 3 5 13 5 3 10 100 10 1 1 1 2 2 1 2 2 2 1 2 3 1 2 1 4 2 4 1 1 2 1 1 1 1 2 1 1 2 1 2 2 2 4 2 4 1 1 1 1 2 2 1 1 1 2 1 1 5 1 2 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 3 25 25 25 25 25 25 100 100 100 100 100 100 25 25 25

In [3]:
print(data_df.dtypes)

Product_Code        object
Warehouse           object
Product_Category    object
Date                object
Order_Demand        object
dtype: object
